# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import xlwings as xw
import numpy as np

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

In [11]:
from xlwings.constants import AutoFillType

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [13]:
import pandas as pd

reviews = pd.read_csv('./07_excel_data/reviews_sample.csv', sep=',', index_col = "Unnamed: 0", parse_dates = ["date"])
recipes = pd.read_csv('./07_excel_data/recipes_sample_with_tags_ingredients.csv', sep=',', index_col = "id", parse_dates = ["submitted"]).drop(columns=["contributor_id", "n_steps", "n_tags", "tags", "ingredients"])
reviews[:3]
#recipes[:3]

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [14]:
some_recipes = recipes.sample(frac = 0.05)
some_reviews = reviews.sample(frac = 0.05)
some_recipes
#some_reviews

,name,minutes,submitted,description,n_ingredients
id,,,,,
159823,peanut dipping sauce with a kick,5,2006-03-13,i searched through the many peanut dipping sau...,13
273270,splenda easy pumpkin pie,105,2007-12-21,you really can't taste the difference in this ...,9
241196,elaine s lasagna,80,2007-07-18,my mom's delicious lasagna recipe.,15
299147,mexican chilli and cheese dip,30,2008-04-17,i first had this in a mexican restaurant in th...,8
270576,quick easy ham chowder,35,2007-12-09,this is a fast meal for those busy weeknights!,7
...,...,...,...,...,...
112180,the lady and sons okra and tomatoes paula deen,55,2005-02-26,"we love okra and tomatoes, this recipe is a lo...",8
508965,homemade smoky veggie bacon salt,10,2013-11-05,this stuff is addictive and the uses are endle...,11
116565,p b j bama cristo,31,2005-04-12,"a comfort food for me, my husband, and kids. m...",6


In [15]:
wb = xw.Book()
wb.save("./07_excel_data/recipes.xlsx")
sheet1 = wb.sheets.add(name = "Рецепты")
sheet1.range("A1").value = some_recipes 
sheet2 = wb.sheets.add(name = "Отзывы")
sheet2.range("A1").value = some_reviews
wb.sheets["Лист1"].delete()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [17]:
wb = xw.Book("./07_excel_data/recipes.xlsx")
sheet = wb.sheets["Рецепты"]
n_rec = str(len(some_recipes)+1)
n_rev = str(len(some_reviews)+1)

seconds_assign = (some_recipes["minutes"] * 60).to_numpy()
sheet.range("G1").value = "seconds_assign"
sheet.range(f"G2:G{n_rec}").options(transpose=True).value = seconds_assign

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [18]:
sheet.range("H1").value = "seconds_formula"
sheet.range("H2").formula = "=C2*60"
sheet.range("H2").api.AutoFill(sheet.range(f"H2:H{n_rec}").api,
                              AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [19]:
sheet.range("I1").value = "n_reviews"
sheet.range("I2").formula = f"=COUNTIF(Отзывы!$C$2:$C${n_rev},"="&A2)"
sheet.range("I2").api.AutoFill(sheet.range(f"I2:I{n_rec}").api,
                              AutoFillType.xlFillDefault)

SyntaxError: can't assign to literal (<ipython-input-19-94caa743e775>, line 2)

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [20]:
sheet.range("G1:I1").api.horizontalalignment = xw.constants.HAlign.xlHAlignCenter
sheet.range("G1:I1").api.Font.Bold = True

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [21]:
for i in range(2, int(n_rec)+1):
    if int(sheet.range(f"C{i}").value) < 5:
        sheet.range(f"C{i}").color = (0, 255, 0)
    elif 5 <= int(sheet.range(f"C{i}").value) <= 10:
        sheet.range(f"C{i}").color = (255, 255, 0)
    elif int(sheet.range(f"C{i}").value) > 10:
        sheet.range(f"C{i}").color = (255, 0, 0)

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [22]:
def validate(sheet):
    sheet.range("H2").formula = f'=COUNTIF(Рецепты!$A$2:Рецепты!$A${n_rec},"="&C2)>0'
    sheet.range("H2").api.AutoFill(sheet.range(f"H2:H{n_rev}").api,
                              AutoFillType.xlFillDefault)
    for i in range(2, int(n_rev)+1):
        rating = int(sheet.range(f'E{i}').value)
        if (rating < 0 or rating > 5) or sheet.range(f'H{i}').value == False:
            sheet.range(f'A{i}:G{i}').color = (255, 0, 0)
    sheet.range('H:H').value = " "
    
validate(wb.sheets["Отзывы"])

In [23]:
wb.save()
wb.close()